In [1]:
import numpy as np
import pandas as pd
import pandas_profiling

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

In [2]:
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
def _classification(x, y, result:list):
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=1 ,stratify=y)
    
    knn = KNeighborsClassifier()
    knn.fit(x_train, y_train)
    res = knn.predict(x_test)
    result.append({'method': 'KNN Classifier', 'score': accuracy_score(res, y_test)})
    
    lda = LinearDiscriminantAnalysis()
    lda.fit(x_train, y_train)
    res = lda.predict(x_test)
    result.append({'method': 'LDA', 'score': accuracy_score(res, y_test)})
    
    qda = QuadraticDiscriminantAnalysis()
    qda.fit(x_train, y_train)
    res = qda.predict(x_test)
    result.append({'method': 'QDA', 'score': accuracy_score(res, y_test)})
    
    log = LogisticRegression()
    log.fit(x_train, y_train)
    res = log.predict(x_test)
    result.append({'method': 'Logisitic Regression Classifier', 'score': accuracy_score(res, y_test)})
    
    dt = DecisionTreeClassifier()
    dt.fit(x_train, y_train)
    res = dt.predict(x_test)
    result.append({'method': 'DT Classifier', 'score': accuracy_score(res, y_test)})
    
    rf = RandomForestClassifier()
    rf.fit(x_train, y_train)
    res = rf.predict(x_test)
    result.append({'method': 'RF Classifier', 'score': accuracy_score(res, y_test)})


In [65]:
le = LabelEncoder()
def _transform(x, columns):
    for column in columns:
        x[column] = le.fit_transform(x[column])
        test[column] = le.transform(test[column])


In [22]:
data = pd.read_csv('train.csv').drop('index', axis=1)
test = pd.read_csv('test.csv').drop('index', axis=1)
y = data['label']
x = data.drop(['label'], axis=1)
# test = test.drop(['Unnamed: 0', 'customerID'], axis=1)
# print(x.shape)
# y = y.drop(x[x.TotalCharges == ' '].index)
# x = x.drop(x[x.TotalCharges == ' '].index)
# print(x.shape)

In [20]:
data['index']

0          0
1          1
2          2
3          3
4          4
        ... 
1432    1792
1433    1793
1434    1794
1435    1795
1436    1796
Name: index, Length: 1437, dtype: int64

In [84]:
columns = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']
_transform(x, columns)

In [78]:
sc = StandardScaler()
x = sc.fit_transform(x, y)
test = sc.transform(test)

In [23]:
result = []
_classification(x, y, result)
print(pd.DataFrame(result))

/Users/therealjamesjung/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/therealjamesjung/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                            method     score
0                   KNN Classifier  0.979167
1                              LDA  0.934028
2                              QDA  0.829861
3  Logisitic Regression Classifier  0.954861
4                    DT Classifier  0.847222
5                    RF Classifier  0.968750


In [31]:
submit = pd.read_csv('sample_submit.csv')
lda = KNeighborsClassifier(n_neighbors=8, weights='distance')
lda.fit(x, y)
res = pd.DataFrame(lda.predict(test), columns=['label'])
submit['label'] = res['label']
submit.to_csv('submit.csv', index=None)